# **Install lib**

In [ ]:
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOW

In [ ]:
# Install thai2transformer 
!pip -q install torch==1.5.0 torchtext==0.4.0 torchvision==0.6.0
!pip -q install transformers==3.4.0 thai2transformers==0.1.2

ERROR: Cannot install thai2transformers==0.1.2 and transformers==3.4.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


# **Import data**

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/Clean_m1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/Clean_m2.csv')
df3 = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/Clean_m3.csv')

In [ ]:
df1.count()

Datetime      37352
Text          37352
CleanTweet    37352
dtype: int64

In [ ]:
df2.count()

Datetime      34384
Text          34384
CleanTweet    34384
dtype: int64

In [ ]:
df3.count()

Datetime      27566
Text          27566
CleanTweet    27566
dtype: int64

In [ ]:
df1['CleanTweet'] = df1['CleanTweet'].str.replace('\u200b', '')
df2['CleanTweet'] = df2['CleanTweet'].str.replace('\u200b', '')
df3['CleanTweet'] = df3['CleanTweet'].str.replace('\u200b', '')

# **Sentiment (Wangchanberta)**

In [ ]:
from tqdm.auto import tqdm
import torch

#datasets
from datasets import load_dataset

#transformers
from transformers import (
    CamembertTokenizer,
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

#thai2transformers
import thai2transformers
from thai2transformers.preprocess import process_transformers
from thai2transformers.metrics import (
    classification_metrics, 
    multilabel_classification_metrics,
)
from thai2transformers.tokenizers import (
    ThaiRobertaTokenizer,
    ThaiWordsNewmmTokenizer,
    ThaiWordsSyllableTokenizer,
    FakeSefrCutTokenizer,
    SEFR_SPLIT_TOKEN
)

In [ ]:
model_names = [
    'wangchanberta-base-att-spm-uncased',
    'xlm-roberta-base',
    'bert-base-multilingual-cased',
    'wangchanberta-base-wiki-newmm',
    'wangchanberta-base-wiki-ssg',
    'wangchanberta-base-wiki-sefr',
    'wangchanberta-base-wiki-spm',
]

tokenizers = {
    'wangchanberta-base-att-spm-uncased': AutoTokenizer,
    'xlm-roberta-base': AutoTokenizer,
    'bert-base-multilingual-cased': AutoTokenizer,
    'wangchanberta-base-wiki-newmm': ThaiWordsNewmmTokenizer,
    'wangchanberta-base-wiki-ssg': ThaiWordsSyllableTokenizer,
    'wangchanberta-base-wiki-sefr': FakeSefrCutTokenizer,
    'wangchanberta-base-wiki-spm': ThaiRobertaTokenizer,
}
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
#@title Choose Pretrained Model
model_name = "wangchanberta-base-att-spm-uncased" #@param ["wangchanberta-base-att-spm-uncased", "xlm-roberta-base", "bert-base-multilingual-cased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm"]

#create tokenizer
tokenizer = tokenizers[model_name].from_pretrained(
                f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

In [ ]:
#@title Choose Multi-class Classification Dataset
dataset_name = "wisesight_sentiment" #@param ['wisesight_sentiment','wongnai_reviews']

#pipeline
classify_multiclass = pipeline(task='sentiment-analysis',
         tokenizer=tokenizer,
         model = f'airesearch/{model_name}' if model_name not in public_models else f'{model_name}',
         revision = f'finetuned@{dataset_name}')

Downloading:   0%|          | 0.00/423M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/716 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

# **Sentiment M1**

In [ ]:
df1_clean = df1['CleanTweet']
df1_clean.count()

37352

In [ ]:
#All data were divided into n sets of 1500 messages each for sentiment analysis.
chunk_size = 1500  #chunk row size
list_df1_clean = [list(df1_clean[i:i+chunk_size]) for i in range(0,df1_clean.shape[0],chunk_size)]

n1 = len(list_df1_clean)
print(n1)

25


In [ ]:
#infer
m1_classify1 = classify_multiclass(list_df1_clean[0])
m1_classify2 = classify_multiclass(list_df1_clean[1])
m1_classify3 = classify_multiclass(list_df1_clean[2])
m1_classify4 = classify_multiclass(list_df1_clean[3])
m1_classify5 = classify_multiclass(list_df1_clean[4])

In [ ]:
#backup_m1_1
m1_classify1_5 = [*m1_classify1, *m1_classify2, *m1_classify3, *m1_classify4, *m1_classify5]
m1_classify1_5_df = pd.DataFrame(m1_classify1_5)
m1_classify1_5_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m1_classify1_5.csv', index=False, header=True)

In [ ]:
m1_classify6 = classify_multiclass(list_df1_clean[5])
m1_classify7 = classify_multiclass(list_df1_clean[6])
m1_classify8 = classify_multiclass(list_df1_clean[7])
m1_classify9 = classify_multiclass(list_df1_clean[8])
m1_classify10 = classify_multiclass(list_df1_clean[9])

In [ ]:
#backup_m1_2
m1_classify6_10 = [*m1_classify6, *m1_classify7, *m1_classify8, *m1_classify9, *m1_classify10]
m1_classify6_10_df = pd.DataFrame(m1_classify6_10)
m1_classify6_10_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m1_classify6_10.csv', index=False, header=True)

In [ ]:
m1_classify11 = classify_multiclass(list_df1_clean[10])
m1_classify12 = classify_multiclass(list_df1_clean[11])
m1_classify13 = classify_multiclass(list_df1_clean[12])
m1_classify14 = classify_multiclass(list_df1_clean[13])
m1_classify15 = classify_multiclass(list_df1_clean[14])

In [ ]:
#backup_m1_3
m1_classify11_15 = [*m1_classify11, *m1_classify12, *m1_classify13, *m1_classify14, *m1_classify15]
m1_classify11_15_df = pd.DataFrame(m1_classify11_15)
m1_classify11_15_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m1_classify11_15.csv', index=False, header=True)

In [ ]:
m1_classify16 = classify_multiclass(list_df1_clean[15])
m1_classify17 = classify_multiclass(list_df1_clean[16])
m1_classify18 = classify_multiclass(list_df1_clean[17])
m1_classify19 = classify_multiclass(list_df1_clean[18])
m1_classify20 = classify_multiclass(list_df1_clean[19])

In [ ]:
#backup_m1_4
m1_classify16_20 = [*m1_classify16, *m1_classify17, *m1_classify18, *m1_classify19, *m1_classify20]
m1_classify16_20_df = pd.DataFrame(m1_classify16_20)
m1_classify16_20_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m1_classify16_20.csv', index=False, header=True)

In [ ]:
m1_classify21 = classify_multiclass(list_df1_clean[20])
m1_classify22 = classify_multiclass(list_df1_clean[21])
m1_classify23 = classify_multiclass(list_df1_clean[22])
m1_classify24 = classify_multiclass(list_df1_clean[23])
m1_classify25 = classify_multiclass(list_df1_clean[24])

In [ ]:
#backup_m1_5
m1_classify21_25 = [*m1_classify21, *m1_classify22, *m1_classify23, *m1_classify24, *m1_classify25]
m1_classify21_25_df = pd.DataFrame(m1_classify21_25)
m1_classify21_25_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m1_classify21_25.csv', index=False, header=True)

In [ ]:
m1_classify = [*m1_classify1_5, *m1_classify6_10, *m1_classify11_15, *m1_classify16_20, *m1_classify21_25]

In [ ]:
len(m1_classify)

37352

In [ ]:
df1_label = pd.DataFrame(m1_classify)

In [ ]:
df1.insert(loc=1,column='label',value=df1_label.label)
df1.insert(loc=2,column='score',value=df1_label.score)

In [ ]:
df1.count()

Datetime      37352
label         37352
score         37352
Text          37352
CleanTweet    37352
dtype: int64

In [ ]:
df1.head()

,Datetime,label,score,Text,CleanTweet
0,2022-01-01 00:01:09+00:00,neu,0.602003,Happy New Year 2022\nขอให้ปีนี้มีแต่เรื่องดีๆ ...,happy<_>new<_>year<_>2022<_>ขอให้ปีนี้มีแต่เรื...
1,2022-01-01 00:05:23+00:00,neu,0.792513,ปีใหม่ขอรัฐบาลใหม่จ้า ขอแบบมาจากการเลือกตั้งที...,ปีใหม่ขอรัฐบาลใหม่จ้า<_>ขอแบบมาจากการเลือกตั้ง...
2,2022-01-01 00:13:06+00:00,neu,0.717748,@redbamboo16 ชัดเจน แจ่มแจ้ง เพียงแค่เรามีรัฐบ...,ชัดเจน<_>แจ่มแจ้ง<_>เพียงแค่เรามีรัฐบาลดีๆ<_>ไ...
3,2022-01-01 00:13:14+00:00,neu,0.541315,Happy new year\n (ปีใหม่แล้ว รัฐบาลใหม่ได้ย...,happy<_>new<_>year<_>(ปีใหม่แล้ว<_>รัฐบาลใหม่ไ...
4,2022-01-01 00:14:34+00:00,neu,0.729572,2022 ขอให้คนไทยได้นายกใหม่กันถ้วนหน้า มีคุณภาพ...,2022<_>ขอให้คนไทยได้นายกใหม่กันถ้วนหน้า<_>มีคุ...


In [ ]:
df1.to_csv('/content/drive/MyDrive/IS/Model_Real/m1_classify.csv', index=False, header=True)

# **Sentiment M2**

In [ ]:
df2_clean = df2['CleanTweet']
df2_clean.count()

34384

In [ ]:
#All data were divided into n sets of 1500 messages each for sentiment analysis.
chunk_size = 1500  #chunk row size
list_df2_clean = [list(df2_clean[i:i+chunk_size]) for i in range(0,df2_clean.shape[0],chunk_size)]

n2 = len(list_df2_clean)
print(n2)

23


In [ ]:
#infer
m2_classify1 = classify_multiclass(list_df2_clean[0])
m2_classify2 = classify_multiclass(list_df2_clean[1])
m2_classify3 = classify_multiclass(list_df2_clean[2])
m2_classify4 = classify_multiclass(list_df2_clean[3])
m2_classify5 = classify_multiclass(list_df2_clean[4])

In [ ]:
#backup_m2_1
m2_classify1_5 = [*m2_classify1, *m2_classify2, *m2_classify3, *m2_classify4, *m2_classify5]
m2_classify1_5_df = pd.DataFrame(m2_classify1_5)
m2_classify1_5_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify1_5.csv', index=False, header=True)

In [ ]:
m2_classify6 = classify_multiclass(list_df2_clean[5])
m2_classify7 = classify_multiclass(list_df2_clean[6])
m2_classify8 = classify_multiclass(list_df2_clean[7])
m2_classify9 = classify_multiclass(list_df2_clean[8])
m2_classify10 = classify_multiclass(list_df2_clean[9])

In [ ]:
#backup_m2_2
m2_classify6_10 = [*m2_classify6, *m2_classify7, *m2_classify8, *m2_classify9, *m2_classify10]
m2_classify6_10_df = pd.DataFrame(m2_classify6_10)
m2_classify6_10_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify6_10.csv', index=False, header=True)

In [ ]:
m2_classify11 = classify_multiclass(list_df2_clean[10])
m2_classify12 = classify_multiclass(list_df2_clean[11])
m2_classify13 = classify_multiclass(list_df2_clean[12])
m2_classify14 = classify_multiclass(list_df2_clean[13])
m2_classify15 = classify_multiclass(list_df2_clean[14])

In [ ]:
#backup_m2_3
m2_classify11_15 = [*m2_classify11, *m2_classify12, *m2_classify13, *m2_classify14, *m2_classify15]
m2_classify11_15_df = pd.DataFrame(m2_classify11_15)
m2_classify11_15_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify11_15.csv', index=False, header=True)

In [ ]:
m2_classify16 = classify_multiclass(list_df2_clean[15])
m2_classify17 = classify_multiclass(list_df2_clean[16])
m2_classify18 = classify_multiclass(list_df2_clean[17])
m2_classify19 = classify_multiclass(list_df2_clean[18])
m2_classify20 = classify_multiclass(list_df2_clean[19])

In [ ]:
#backup_m2_4
m2_classify16_20 = [*m2_classify16, *m2_classify17, *m2_classify18, *m2_classify19, *m2_classify20]
m2_classify16_20_df = pd.DataFrame(m2_classify16_20)
m2_classify16_20_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify16_20.csv', index=False, header=True)

In [ ]:
m2_classify21 = classify_multiclass(list_df2_clean[20])
m2_classify22 = classify_multiclass(list_df2_clean[21])
m2_classify23 = classify_multiclass(list_df2_clean[22])

In [ ]:
#backup_m2_5
m2_classify21_23 = [*m2_classify21, *m2_classify22, *m2_classify23]
m2_classify21_23_df = pd.DataFrame(m2_classify21_23)
m2_classify21_23_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify21_23.csv', index=False, header=True)

In [ ]:
#session crashed while running data set 21-23
#fix problem : use m2_classify1_5, m2_classify6_10, m2_classify11_15, m2_classify16_20 from backup
m2_classify1_5_df = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify1_5.csv')
m2_classify6_10_df = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify1_5.csv')
m2_classify11_15_df = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify1_5.csv')
m2_classify16_20_df = pd.read_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m2_classify1_5.csv')

m2_classify1_5 = m2_classify1_5_df.to_dict('records')
m2_classify6_10 = m2_classify6_10_df.to_dict('records')
m2_classify11_15 = m2_classify11_15_df.to_dict('records')
m2_classify16_20 = m2_classify16_20_df.to_dict('records')

In [ ]:
m2_classify = [*m2_classify1_5, *m2_classify6_10, *m2_classify11_15, *m2_classify16_20, *m2_classify21_23]

In [ ]:
len(m2_classify)

34384

In [ ]:
df2_label = pd.DataFrame(m2_classify)

In [ ]:
df2.insert(loc=1,column='label',value=df2_label.label)
df2.insert(loc=2,column='score',value=df2_label.score)

In [ ]:
df2.count()

Datetime      34384
label         34384
score         34384
Text          34384
CleanTweet    34384
dtype: int64

In [ ]:
df2.head()

,Datetime,label,score,Text,CleanTweet
0,2022-02-01 00:01:18+00:00,neg,0.972794,โอ้ยยยย อีเหี้ย ชีวิตที่เมืองไทย ถ้ากุยังอยุ่เ...,โอ้ย<_>อีเหี้ย<_>ชีวิตที่เมืองไทย<_>ถ้ากุยังอย...
1,2022-02-01 00:01:58+00:00,neg,0.614448,@ktnewsonline สังเกตนะครับ พรรคที่ชนะเลือกตั้ง...,สังเกตนะครับ<_>พรรคที่ชนะเลือกตั้งปี<_>62<_>แต...
2,2022-02-01 00:04:38+00:00,neg,0.706246,ไปเมืองท่องเที่ยวเช่ารถเอานะ \nไม่ต้องไปใช้หรอ...,ไปเมืองท่องเที่ยวเช่ารถเอานะ<_>ไม่ต้องไปใช้หรอ...
3,2022-02-01 00:08:40+00:00,neg,0.983391,ชาวบ้านต้องพึ่งปลัดขิกละ อีรัฐบาลช่วยไรไม่ได้ ...,ชาวบ้านต้องพึ่งปลัดขิกละ<_>อีรัฐบาลช่วยไรไม่ได...
4,2022-02-01 00:11:46+00:00,neu,0.542339,@taopiphop เขารู้กันจ้า แต่เป็นนโยบายรัฐบาลนี้...,เขารู้กันจ้า<_>แต่เป็นนโยบายรัฐบาลนี้จ้า<_>เข้...


In [ ]:
df2.to_csv('/content/drive/MyDrive/IS/Model_Real/m2_classify.csv', index=False, header=True)

# **Sentiment M3**

In [ ]:
df3_clean = df3['CleanTweet']
df3_clean.count()

27566

In [ ]:
chunk_size = 1500  #chunk row size
list_df3_clean = [list(df3_clean[i:i+chunk_size]) for i in range(0,df3_clean.shape[0],chunk_size)]

n3 = len(list_df3_clean)
print(n3)

19


In [ ]:
#infer
m3_classify1 = classify_multiclass(list_df3_clean[0])
m3_classify2 = classify_multiclass(list_df3_clean[1])
m3_classify3 = classify_multiclass(list_df3_clean[2])
m3_classify4 = classify_multiclass(list_df3_clean[3])
m3_classify5 = classify_multiclass(list_df3_clean[4])

In [ ]:
#backup_m3_1
m3_classify1_5 = [*m3_classify1, *m3_classify2, *m3_classify3, *m3_classify4, *m3_classify5]
m3_classify1_5_df = pd.DataFrame(m3_classify1_5)
m3_classify1_5_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m3_classify1_5.csv', index=False, header=True)

In [ ]:
m3_classify6 = classify_multiclass(list_df3_clean[5])
m3_classify7 = classify_multiclass(list_df3_clean[6])
m3_classify8 = classify_multiclass(list_df3_clean[7])
m3_classify9 = classify_multiclass(list_df3_clean[8])
m3_classify10 = classify_multiclass(list_df3_clean[9])

In [ ]:
#backup_m3_2
m3_classify6_10 = [*m3_classify6, *m3_classify7, *m3_classify8, *m3_classify9, *m3_classify10]
m3_classify6_10_df = pd.DataFrame(m3_classify6_10)
m3_classify6_10_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m3_classify6_10.csv', index=False, header=True)

In [ ]:
m3_classify11 = classify_multiclass(list_df3_clean[10])
m3_classify12 = classify_multiclass(list_df3_clean[11])
m3_classify13 = classify_multiclass(list_df3_clean[12])
m3_classify14 = classify_multiclass(list_df3_clean[13])
m3_classify15 = classify_multiclass(list_df3_clean[14])

In [ ]:
#backup_m3_3
m3_classify11_15 = [*m3_classify11, *m3_classify12, *m3_classify13, *m3_classify14, *m3_classify15]
m3_classify11_15_df = pd.DataFrame(m3_classify11_15)
m3_classify11_15_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m3_classify11_15.csv', index=False, header=True)

In [ ]:
m3_classify16 = classify_multiclass(list_df3_clean[15])
m3_classify17 = classify_multiclass(list_df3_clean[16])
m3_classify18 = classify_multiclass(list_df3_clean[17])
m3_classify19 = classify_multiclass(list_df3_clean[18])

In [ ]:
#backup_m3_4
m3_classify16_19 = [*m3_classify16, *m3_classify17, *m3_classify18, *m3_classify19]
m3_classify16_19_df = pd.DataFrame(m3_classify16_19)
m3_classify16_19_df.to_csv('/content/drive/MyDrive/IS/Model_Real/backup_classify/m3_classify16_19.csv', index=False, header=True)

In [ ]:
m3_classify = [*m3_classify1_5, *m3_classify6_10, *m3_classify11_15, *m3_classify16_19]

In [ ]:
len(m3_classify)

27566

In [ ]:
df3_label = pd.DataFrame(m3_classify)

In [ ]:
df3.insert(loc=1,column='label',value=df3_label.label)
df3.insert(loc=2,column='score',value=df3_label.score)

In [ ]:
df3.count()

Datetime      27566
label         27566
score         27566
Text          27566
CleanTweet    27566
dtype: int64

In [ ]:
df3.head()

,Datetime,label,score,Text,CleanTweet
0,2022-03-01 00:00:29+00:00,neu,0.679244,@WassanaNanuam 3.ทีนี้มาแก้ตัวแบบไม่อายสุนัขว่...,3.ทีนี้มาแก้ตัวแบบไม่อายสุนัขว่ารัฐบาลเยอรมนีเ...
1,2022-03-01 00:02:06+00:00,neg,0.508819,ได้ยินมาว่ารัฐเริ่มเรียกเก็บภาษีย้อนหลังกันแล้...,ได้ยินมาว่ารัฐเริ่มเรียกเก็บภาษีย้อนหลังกันแล้...
2,2022-03-01 00:05:53+00:00,neg,0.935615,รพ เอกชนวันนี้ ไม่ตรวจฟรี ไม่รักษาฟรีแล้ว รัฐบ...,รพ<_>เอกชนวันนี้<_>ไม่ตรวจฟรี<_>ไม่รักษาฟรีแล้...
3,2022-03-01 00:06:13+00:00,neu,0.736409,@MatichonOnline ไม่ต้องพูดก็รู้ว่ารัฐบาลทหารจะ...,ไม่ต้องพูดก็รู้ว่ารัฐบาลทหารจะเชียร์ใครถ้าไม่ใ...
4,2022-03-01 00:12:00+00:00,neu,0.788561,เอาจริง ๆ การที่รัฐบาลยูเครนอนุญาติให้พลเรือนช...,เอาจริง<_>ๆ<_>การที่รัฐบาลยูเครนอนุญาติให้พลเร...


In [ ]:
df3.to_csv('/content/drive/MyDrive/IS/Model_Real/m3_classify.csv', index=False, header=True)